In [1]:
import numpy as np
import torch as T
device = 'cpu'

In [2]:
def forward(x, w, b):
    z = T.dot(x, w).reshape(1)
    z += b
    p = 1 / (1 + T.exp(-z))
    return p

In [3]:
seed = 0xEDA

In [4]:
print("Создадим данные")
T.manual_seed(seed)
np.random.seed(seed)

# 1. Модельные данные для обучения
train_x = np.array([
    [1.5, 2.5], [3.5, 4.5], [6.5, 6.5],
    [4.5, 1.5], [5.5, 3.5], [7.5, 5.5]], dtype=np.float32)
train_y = np.array([1, 1, 1, 0, 0, 0], dtype=np.long)

Создадим данные


In [5]:
print("Превратим в тензоры:")
print(train_x)
print(train_y)

train_x = T.tensor(train_x, dtype=T.float32).to(device)
train_y = T.tensor(train_y, dtype=T.long).to(device)

Превратим в тензоры:
[[1.5 2.5]
 [3.5 4.5]
 [6.5 6.5]
 [4.5 1.5]
 [5.5 3.5]
 [7.5 5.5]]
[1 1 1 0 0 0]


In [6]:
# границы для случайной величины
lo = -0.01; hi = 0.01
w = T.rand((2), dtype=T.float32, requires_grad=True).to(device)
w = (hi - lo) * w + lo
w.grad = T.zeros(2)
w.retain_grad()

b = T.zeros((1), dtype=T.float32, requires_grad=True).to(device)
b.grad = T.zeros(1)
b.retain_grad()

In [12]:
print("Обучение")

lrn_rate = 0.05
indices = np.arange(6)

for epoch in range(0, 100):
    tot_loss = 0
    tot_loss = T.zeros((1), dtype=T.float32, requires_grad=True).to(device)
    tot_loss.grad = T.zeros(1)
    tot_loss.retain_grad()
    
    np.random.shuffle(indices)  
    for ii in range(len(indices)):
        i = indices[ii]
        x = train_x[i]
        target = train_y[i]
        oupt = forward(x, w, b)
        loss = (oupt - target).pow(2).sum()
        tot_loss = loss+tot_loss
        
#     tot_loss = tot_loss + T.norm(w, p=2) # l2 reg
#     tot_loss = tot_loss + T.norm(w, p=1) # l1 reg

    tot_loss.backward(retain_graph=True)  # compute gradients

    w.data += -1 * lrn_rate * w.grad.data
    b.data += -1 * lrn_rate * b.grad.data

    w.grad = T.zeros(2)
    b.grad = T.zeros(1)

    if epoch % 10 == 0:
        print("epoch = %4d " % epoch, end="")
        print("   loss = %6.4f" % (tot_loss / 6))

Обучение
epoch =    0    loss = 0.1489
epoch =   10    loss = 0.0835
epoch =   20    loss = 0.0572
epoch =   30    loss = 0.0441
epoch =   40    loss = 0.0358
epoch =   50    loss = 0.0301
epoch =   60    loss = 0.0259
epoch =   70    loss = 0.0228
epoch =   80    loss = 0.0203
epoch =   90    loss = 0.0182
